In [7]:
import json
import pandas as pd
json_path = 'D:/毕业设计/inference/val.json'
with open(json_path, 'r', encoding='UTF-8') as file_in:
  file = json.load(file_in)
  print(file)
  results = file["results"]
  print(results)
  top1 = 0
  top5 = 0
  highRate = 0
  for key in results:
    list = results[key]
    keyword = key.split('_')[1]
    for i in range(5):
      if i == 0:
        top1 += (list[i]["label"] == keyword)
        highRate += (list[i]["label"] == keyword) & (list[i]["score"] >= 0.9)
      top5 += (list[i]["label"] == keyword)
  print('top1: '+str(top1))
  print('top5: '+str(top5))
  print('highRate: ' + str(highRate))
  

{'results': {'v_ApplyEyeMakeup_g01_c02': [{'label': 'ApplyEyeMakeup', 'score': 0.9996572732925415}, {'label': 'ApplyLipstick', 'score': 0.0003379936679266393}, {'label': 'ShavingBeard', 'score': 2.5395538614247926e-06}, {'label': 'BrushingTeeth', 'score': 1.3170912325222162e-06}, {'label': 'PlayingGuitar', 'score': 3.4234537338306836e-07}], 'v_ApplyEyeMakeup_g02_c01': [{'label': 'ApplyEyeMakeup', 'score': 0.9744638800621033}, {'label': 'ShavingBeard', 'score': 0.01380712166428566}, {'label': 'BrushingTeeth', 'score': 0.010451249778270721}, {'label': 'ApplyLipstick', 'score': 0.0006468567298725247}, {'label': 'HeadMassage', 'score': 0.0005341463256627321}], 'v_ApplyEyeMakeup_g03_c01': [{'label': 'ApplyEyeMakeup', 'score': 0.7759988903999329}, {'label': 'ApplyLipstick', 'score': 0.22161275148391724}, {'label': 'BlowDryHair', 'score': 0.0017847991548478603}, {'label': 'BrushingTeeth', 'score': 0.00015205082308966666}, {'label': 'Typing', 'score': 0.00011664883641060442}], 'v_ApplyEyeMakeu

导入依赖

In [4]:
import cv2
import os
video_path = 'E:/train_data/UCF101/UCF-101'

图像名生成规则函数

In [2]:
def image_name_formatter(x):
    return f'image_{x:05d}.jpg'

保留一部分视频，减小数据集

In [10]:
action_list = os.listdir(video_path)
# 保存每个动作集的前4个视频
save_map = ['g01','g02','g03','g04']
for action in action_list:
  # 视频列表
  video_list = os.listdir(video_path+'/'+action)
  for video in video_list:
    if video.split('_')[2] not in save_map:
      os.remove(video_path+'/'+action+'/'+video)

预处理视频生成每一帧的图片

In [11]:
# 生成的图像数据集存储位置
save_path = 'E:/resnet_data/jpgs'
# 如果文件路径不存在则创建路径
if not os.path.exists(save_path):
    os.mkdir(save_path)
# 获取动作列表
action_list = os.listdir(video_path)
# 遍历所有动作
for action in action_list:
    if action.startswith(".")==False:
        if not os.path.exists(save_path+'/'+action):
            os.mkdir(save_path+'/'+action)
        video_list = os.listdir(video_path+'/'+action)
        #遍历所有视频
        for video in video_list:
            prefix = video.split('.')[0]
            if not os.path.exists(os.path.join(save_path, action, prefix)):
                os.mkdir(os.path.join(save_path, action, prefix))
            save_name = os.path.join(save_path, action, prefix) + '/'
            video_name = video_path+'/'+action+'/'+video
            # 读取视频文件
            # cap为视频的帧
            cap = cv2.VideoCapture(video_name)
            # fps为帧率
            fps = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            fps_count = 0
            for i in range(fps):
                ret, frame = cap.read()
                if ret:
                    # 将帧画面写入图片文件中
                    cv2.imwrite(save_name+image_name_formatter(fps_count),frame)
                    fps_count += 1

In [12]:
video_list = os.listdir(video_path+'/'+action)
print(video_list)

['v_YoYo_g01_c01.avi', 'v_YoYo_g01_c02.avi', 'v_YoYo_g01_c03.avi', 'v_YoYo_g01_c04.avi', 'v_YoYo_g01_c05.avi', 'v_YoYo_g01_c06.avi', 'v_YoYo_g01_c07.avi', 'v_YoYo_g02_c01.avi', 'v_YoYo_g02_c02.avi', 'v_YoYo_g02_c03.avi', 'v_YoYo_g02_c04.avi', 'v_YoYo_g02_c05.avi', 'v_YoYo_g03_c01.avi', 'v_YoYo_g03_c02.avi', 'v_YoYo_g03_c03.avi', 'v_YoYo_g03_c04.avi', 'v_YoYo_g03_c05.avi', 'v_YoYo_g03_c06.avi', 'v_YoYo_g04_c01.avi', 'v_YoYo_g04_c02.avi', 'v_YoYo_g04_c03.avi', 'v_YoYo_g04_c04.avi', 'v_YoYo_g04_c05.avi']


In [13]:
# 保存数据
def writeData(datalist,savepath):
    index = 1
    for data in datalist:
        with open(savepath, mode="a", encoding="utf-8") as f:
            f.write(str(index) + ' ' + data)   #写数据
            f.write("\n")   #换行
        index = index + 1
    f.close()

In [5]:
img_path="E:/resnet_data/jpgs"
cates=os.listdir(img_path)
writeData(cates,"E:/resnet_data/txt/classInd.txt")

In [1]:
# 读取类别词典
def read_classes(path):
    with open(path, "r+") as f:
        lines = f.readlines()
    classes = {}
    for line in lines:
        c_id = line.split()[0]
        c_name = line.split()[1]
        classes[c_name] =c_id 
    return classes
classes_dict = read_classes("E:/resnet_data/txt/classInd.txt")
print(classes_dict)

{'ApplyEyeMakeup': '1', 'ApplyLipstick': '2', 'Archery': '3', 'BabyCrawling': '4', 'BalanceBeam': '5', 'BandMarching': '6', 'BaseballPitch': '7', 'Basketball': '8', 'BasketballDunk': '9', 'BenchPress': '10', 'Biking': '11', 'Billiards': '12', 'BlowDryHair': '13', 'BlowingCandles': '14', 'BodyWeightSquats': '15', 'Bowling': '16', 'BoxingPunchingBag': '17', 'BoxingSpeedBag': '18', 'BreastStroke': '19', 'BrushingTeeth': '20', 'CleanAndJerk': '21', 'CliffDiving': '22', 'CricketBowling': '23', 'CricketShot': '24', 'CuttingInKitchen': '25', 'Diving': '26', 'Drumming': '27', 'Fencing': '28', 'FieldHockeyPenalty': '29', 'FloorGymnastics': '30', 'FrisbeeCatch': '31', 'FrontCrawl': '32', 'GolfSwing': '33', 'Haircut': '34', 'Hammering': '35', 'HammerThrow': '36', 'HandStandPushups': '37', 'HandstandWalking': '38', 'HeadMassage': '39', 'HighJump': '40', 'HorseRace': '41', 'HorseRiding': '42', 'HulaHoop': '43', 'IceDancing': '44', 'JavelinThrow': '45', 'JugglingBalls': '46', 'JumpingJack': '47', 'J

训练数据和测试数据生成

In [6]:
import random
train_list = []
test_list = []
for cate in cates:
  videos = os.listdir(os.path.join(img_path,cate))
  length = len(videos)//5
  # 训练集大小，随机取视频文件加入训练集
  train= random.sample(videos, length*4)
  train_list.extend(train)
  # 将余下的视频加入测试集
  for video in videos:
      if video not in train:
          test_list.append(video)
print(train_list)

['v_ApplyEyeMakeup_g01_c01', 'v_ApplyEyeMakeup_g03_c02', 'v_ApplyEyeMakeup_g02_c03', 'v_ApplyEyeMakeup_g04_c06', 'v_ApplyEyeMakeup_g02_c01', 'v_ApplyEyeMakeup_g01_c03', 'v_ApplyEyeMakeup_g04_c03', 'v_ApplyEyeMakeup_g03_c04', 'v_ApplyEyeMakeup_g02_c04', 'v_ApplyEyeMakeup_g03_c03', 'v_ApplyEyeMakeup_g01_c04', 'v_ApplyEyeMakeup_g04_c02', 'v_ApplyEyeMakeup_g01_c05', 'v_ApplyEyeMakeup_g03_c06', 'v_ApplyEyeMakeup_g03_c05', 'v_ApplyEyeMakeup_g04_c04', 'v_ApplyLipstick_g02_c03', 'v_ApplyLipstick_g01_c02', 'v_ApplyLipstick_g02_c02', 'v_ApplyLipstick_g01_c05', 'v_ApplyLipstick_g02_c01', 'v_ApplyLipstick_g04_c02', 'v_ApplyLipstick_g04_c03', 'v_ApplyLipstick_g03_c01', 'v_ApplyLipstick_g03_c04', 'v_ApplyLipstick_g01_c01', 'v_ApplyLipstick_g02_c04', 'v_ApplyLipstick_g04_c04', 'v_Archery_g04_c03', 'v_Archery_g02_c02', 'v_Archery_g03_c01', 'v_Archery_g04_c04', 'v_Archery_g03_c03', 'v_Archery_g02_c07', 'v_Archery_g04_c05', 'v_Archery_g01_c07', 'v_Archery_g01_c06', 'v_Archery_g03_c02', 'v_Archery_g04_c0

In [7]:
def writeTestList(datalist,savepath):
  for data in datalist:
    with open(savepath, mode="a", encoding="utf-8") as f:
      item = data.split('_')[1]
      f.write(item + '/' + data + '.avi')   #写数据
      f.write("\n")   #换行
  f.close()

def writeTrainList(datalist,savepath,classes_dict):
  for data in datalist:
    with open(savepath, mode="a", encoding="utf-8") as f:
      item = data.split('_')[1]
      f.write(item + '/' + data + '.avi' + ' ' +classes_dict[item])   #写数据
      f.write("\n")   #换行
  f.close()

In [19]:
writeTrainList(train_list,"E:/resnet_data/txt/trainlist01.txt",classes_dict)
writeTestList(test_list,"E:/resnet_data/txt/testlist01.txt")

In [1]:
list = [1,2,3,4,5,6]
print(list[-1])

6


生成json文件

In [8]:
!python -m util_scripts.ucf101_json "E:/resnet_data/txt" "E:/resnet_data/jpgs" "E:/resnet_data/json"

**temporal_transforms.py**  用于处理传入的某一个视频帧 生成对应opt.sample_duration的某一组视频帧  时间变换(选取视频帧)  
**详细解释(仅仅解释目前所用到的类)**  
Compose类: 首先需要注意的是__call__函数，其作用是可以使得对象可以当作函数一样进行调用，即
``` python
compose = Compose(transforms) # 调用__init__
compose(frame_indices) # 调用__call__ 其中frame_indices为传入的是视频帧 该call函数的作用是执行transform.compose中的一组图片变换操作
```

**spatial_transforms.py**  用于定义图片转换方法类别的py文件
**loader.py**  用于定义VideoLoader的py文件 其中__call__是将视频帧中的所有图片放入video中并且返回
**videodataset.py** 用于生成videodataset的py文件 
``` python
video_path_formatter = (
            lambda root_path, label, video_id: root_path / label / video_id)
__getitem__ # 调用时机为使用[]读取时调用
```
**main.py**  
get_val_utils()